# Hyperparameter tuning with HyperEstimator

This notebook contains the code to tune hyperparameters for a given for Convolutional Neural Networks and set of hyerparameters.

In this set of experiments, the hyperparameters used are: {optimiser, learning rate, batch size} for the CNN architecture for image classification. The CNN model used is pretrained using VGG-16 model with pretrained weights.

The notebook requires the structure of the folder to be maintained as it is in the respository. 

The prerequisites are mentioned in the requirements.txt and can be installed by using the command:

pip install -r requirements.txt

All the requirements are satisfied with Anaconda.

The notebook executes with the following steps:

1. Generating candidate solutions with Grammatical Evolution
2. Estimating the hyperparameters with Linear Regression and Bayesian Optimiser

# Importing all the required libraries

In [ ]:
# import packages 
import numpy as np 
import pandas as pd 
from sklearn.ensemble import RandomForestClassifier 
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler 
from hyperopt import tpe, hp, fmin, STATUS_OK,Trials, space_eval
from hyperopt.pyll.base import scope
import matplotlib.gridspec as gridspec
import pandas as pd
from sklearn import linear_model
import statsmodels.api as sm

import warnings
warnings.filterwarnings("ignore")

# Generating candidate solutions with Grammatical Evolution

This experiment will run the grammatical evolution 

In [ ]:
! scripts/experiments_manager.py --parameters final_parameters --runs 10 --experiment_name hyperestimator

# Regressor Model

In [110]:
# hyperestimator.csv file in contained in src/ folder. In case you are running all the files seperately, replace the appropriate path.
df = pd.read_csv('hyperestimator.csv')
df.columns =['opt', 'lr', 'batch_size', 'val_acc']
X = df[['opt','lr', 'batch_size']] 
Y = df['val_acc']
 
# with sklearn
regr = linear_model.LinearRegression()
regr.fit(X, Y)

print('Intercept: \n', regr.intercept_)
print('Coefficients: \n', regr.coef_)

# with statsmodels
X = sm.add_constant(X)
 
model = sm.OLS(Y, X).fit()

Intercept: 
 0.6068699182325434
Coefficients: 
 [-3.01706693e-02 -3.64305652e+02 -3.34956686e-04]


# Bayesian Optimiser

In [111]:
def hyperparameter_tuning(params):
    lr = params['lr']
    opt = params['opt']
    bs = params['bs']
    # replace the coefficients in the equation obtained from Linear Regression model
    f1 = (-3.18194782e-02 * opt) + (-3.80413117e+02 * lr) + (-3.09998001e-04 * bs) +  0.6085946681414556
    return (1/f1)

In [112]:
#define the hyperparameter search space for the 
space = {
"opt": hp.choice("opt",[0, 1]), #0: adam,1: RMSProp 
"lr": hp.choice("lr",[0.000001, 0.00001, 0.0001, 0.001]),
"bs": hp.choice("bs",[16, 32,64, 128, 256]),
}

In [113]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
trials = Trials()
best = fmin(fn=hyperparameter_tuning, space=space, algo=tpe.suggest, max_evals=1000, trials=trials)
print ('best:', best)

100%|████████████████████████████████████████████| 1000/1000 [00:15<00:00, 65.72trial/s, best loss: 1.6576757455947981]
best: {'bs': 0, 'lr': 0, 'opt': 0}
